In [1]:

%load_ext autoreload
%autoreload 2

Procedure

- Create mesh
    - Generates object with nodes and elements as properties
    - Depends on coordinate system, mesh size, and symmetry conditions

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

- Create mesh 
    - Depends on element size and extents
    - Returns coordinate axes (size of each is nx, ny, nz)
- Add structures to mesh
    - Get nodes in struct based on coordinates
    - Change part id of elements in struct
        - All nodes in element should be within struct? 
    -- Generate list of node ids in struct
- Add boundary constraints 
    - Depends on mesh symmetry
    - Change tc and rc of nodes on faces and edges
- Add pml
    - Depends on mesh symmetry and pml thickness
    - Change part id of non-symmetry plane elements
    - Part id + 1 if also in struct
- Finish dyna deck
    - Add material
    - Add load curves
    - Add arf load based on field params (pre-generated)
    - Add database/control parameters
        - Node sets?
- Validation functions
    - Make sure nodes exist on center axis regardless of mesh symmetry so peak arf load is modeled



In [5]:
import fem
import pathlib

from fem.dyna.mesh import Coordinates, DynaMesh
from fem.dyna._structure import Structure
from fem.dyna.material import KelvinMaxwellViscoelastic

project_path = pathlib.Path(fem.__file__).parents[1]
sims_path = project_path.joinpath('sims')
sims_path.mkdir(parents=True, exist_ok=True)

mat1 = KelvinMaxwellViscoelastic(density=1, E=26.11, nu=0.499, eta=2.34)
mat2 = KelvinMaxwellViscoelastic(density=1, E=2*26.11, nu=0.499, eta=2.34)

struct_list = [
    Structure(
        shape = 'rectangle', 
        args = [-1, -0.5, 0, 0.5, -4, -2],
        material = mat2,  
    ), 
    # Structure(
    #     shape = 'sphere', 
    #     args = [0, 0, -2, 1],
    #     material = mat2,  
    # ), 
]

coords = Coordinates(
    nx=51, ny=51, nz=101,
    xmin=-1.0, xmax=0.0,  # ele
    ymin=0.0, ymax=0.5,  # lat
    zmin=-4.0, zmax=-0.15   # ax
)
symmetry = 'q'
mesh = DynaMesh(coords, symmetry, mat1)

mesh.constrain_boundary_nodes()
# mesh.add_pml(pml_thickness=2, exclude_faces=['zmin'])
# mesh.add_pml(pml_thickness=2)
# mesh.add_struct_list(struct_list)

mesh.set_control(end_time=4.5e-3)
mesh.set_database(dt=2e-5)
mesh.set_master(title='testing')

load_curve_id = 1
normalization_isppa = 1000
field_load_file = project_path.joinpath('loads/test_loads.mat')
mesh.add_load_curve(load_curve_id, 'arf', [70e-6, 5e-6])
mesh.add_field_arf_load(field_load_file, normalization_isppa, load_curve_id)

load_folder_name = 'load_0'
material_folder_name = 'material_0'
mesh.write_all_dyna_cards(sims_path, load_folder_name, material_folder_name)
